<!-- docusaurus_head_meta::start
---
title: Introduction Notebook
---
docusaurus_head_meta::end -->

<div style="text-align: center;">
    <img src="http://wandb.me/logo-im-png" width="600" alt="Weights & Biases" />
</div>
<!--- @wandbcode{intro-colab} -->

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:



* [docs: Weave](https://weave-docs.wandb.ai/)


* [Introduction Weave](https://colab.research.google.com/github/wandb/weave/blob/master/docs/intro_notebook.ipynb?_gl=1%2A1lgxu63%2A_gcl_au%2AMTEwMzA2ODYwNi4xNzI4MTkwMjUy%2A_ga%2AMTQ0OTQ4MDk0MC4xNzI4MTg4NjEz%2A_ga_JH1SJHJQXJ%2AMTcyODYxODQzNS4zLjEuMTcyODYxODQ4MS4xNC4wLjA.%2A_ga_GMYDGNGKDT%2AMTcyODYxODQzNS4zLjEuMTcyODYxODQ4MS4wLjAuMA..#scrollTo=5ef6f27b)





`Weave`

Weave é um kit de ferramentas leve para rastrear e avaliar aplicações de LLM, criado pela Weights & Biases.

# 🏃‍♀️ Quickstart

Comece a usar o `Weave` para:
- Registrar (log) e depurar entradas, saídas e rastros do modelo de linguagem
- Crie avaliações rigorosas e comparativas para casos de uso de modelos de linguagem
- Organizar todas as informações geradas no fluxo de trabalho do LLM, desde a experimentação até as avaliações e a produção

Veja a documentação completa do Weave [aqui](https://wandb.me/weave).


## 🪄 Install `weave` library and login


Comece instalando a biblioteca e fazendo login na sua conta.

Neste exemplo, estamos usando o Openai, então você deve [adicionar uma chave de API da OpenAI](https://platform.openai.com/docs/quickstart/step-2-setup-your-api-key).


In [1]:
%%capture
#%pip install weave openai set-env-colab-kaggle-dotenv
%pip install weave openai


In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
#Eddy_key_openai  = os.environ['OPENAI_API_KEY']

#from openai import OpenAI
#client = OpenAI(api_key=Eddy_key_openai)

PROJECT = "weave-intro-notebook"

# Track inputs & outputs of functions

Weave allows users to track function calls: the code, inputs, outputs, and even LLM tokens & costs! In the following sections we will cover:

* Custom Functions
* Vendor Integrations
* Nested Function Calling
* Error Tracking

Note: in all cases, we will:

```python
import weave                    # import the weave library
weave.init('project-name')      # initialize tracking for a specific W&B project
```

## Track custom functions

Add the `@weave.op` decorator to the functions you want to track

![](https://github.com/wandb/weave/blob/media/intro/1.png?raw=1)

In [2]:
from openai import OpenAI

import weave

weave.init(PROJECT)

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "Você é um corretor gramatical, corrija a seguinte entrada do usuário.",
        },
        {"role": "user", "content": "Eui estouu indo para o cinema a assistir um filme!"},
    ],
    temperature=0,
)
generation = response.choices[0].message.content
print(generation)

weave version 0.51.14 has been recalled!  (Evaluation framework is mis-reporting the inputs for prediction records)  Please upgrade.
Logged in as Weights & Biases user: eddygiusepe.
View Weave data at https://wandb.ai/eddygiusepe/weave-intro-notebook/weave
🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf2b-e7af-7f52-b2aa-ab42e95d5bfe
Eu estou indo para o cinema assistir a um filme!


You can find your interactive dashboard by clicking any of the  👆 wandb links above.

## Vendor Integrations (OpenAI, Anthropic, Mistral, etc...)

Aqui, estamos rastreando automaticamente todas as chamadas para `openai`. Rastreamos automaticamente muitas bibliotecas LLM, mas é muito fácil adicionar suporte para qualquer LLM que você esteja usando, como você verá abaixo.

In [3]:
import weave

weave.init(PROJECT)


@weave.op()
def strip_user_input(user_input):
    return ' '.join(user_input.split()) # Isto tira todos os espaços
    #return user_input.strip()  < -- Essa função só tira os espaços de início e fim    

result = strip_user_input("   Olá,      Eddy    ")
print(result)

🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf2c-0660-7890-9ba5-057edd80f598
Olá, Eddy


After adding `weave.op` and calling the function, visit the link and see it tracked within your project.

💡 We automatically track your code, have a look at the code tab!

## Track nested functions


Agora que você viu o básico, vamos combinar tudo acima e rastrear algumas funções profundamente aninhadas junto com chamadas LLM.

In [4]:
from openai import OpenAI
import weave

weave.init(PROJECT)


@weave.op()
def strip_user_input(user_input):
    return ' '.join(user_input.split())


@weave.op()
def correct_grammar(user_input):
    client = OpenAI()

    stripped = strip_user_input(user_input)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Você é um corretor gramatical, corrija a seguinte entrada do usuário.",
            },
            {"role": "user", "content": stripped},
        ],
        temperature=0,
    )
    return response.choices[0].message.content



result = correct_grammar("   Eui           estouu indo para o cinema a               assistir um fi   lme!    ")
print(result)

🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf34-466c-70a2-9403-e6e8bab20f51
Eu estou indo para o cinema assistir a um filme!


## Track Errors

Sempre que seu código travar, o weave destacará o que causou o problema. Isso é especialmente útil para encontrar coisas como problemas de análise de JSON que podem acontecer ocasionalmente ao analisar dados de respostas LLM.

In [5]:
import json

from openai import OpenAI

import weave

weave.init(PROJECT)


@weave.op()
def strip_user_input(user_input):
    return ' '.join(user_input.split())


@weave.op()
def correct_grammar(user_input):
    client = OpenAI()

    stripped = strip_user_input(user_input)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "Você é um corretor gramatical, corrija a seguinte entrada do usuário.",
            },
            {"role": "user", "content": stripped},
        ],
        temperature=0,
        response_format={"type": "json_object"},
    )
    return json.loads(response.choices[0].message.content)


result = correct_grammar("   Eui           estouu indo para o cinema a               assistir um fi   lme!    ")
print(result)

🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf39-e9e3-79e3-bed5-3169f0cb162e


BadRequestError: Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'.", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

# Tracking Objects

Organizing experimentation is difficult when there are many moving pieces. You can capture and organize the experimental details of your app like your system prompt or the model you're using within `weave.Objects`. This helps organize and compare different iterations of your app. In this section, we will cover:

* General Object Tracking
* Tracking Models
* Tracking Datasets

## General Object Tracking

Many times, it is useful to track & version data, just like you track and version code. For example, here we define a `SystemPrompt(weave.Object)` object that can be shared between teammates

![](https://github.com/wandb/weave/blob/media/intro/5.png?raw=1)

In [6]:
import weave

weave.init(PROJECT)


class SystemPrompt(weave.Object):
    prompt: str


system_prompt = SystemPrompt(
    prompt="Você é um corretor gramatical, corrija a seguinte entrada do usuário."
)

weave.publish(system_prompt)

📦 Published to https://wandb.ai/eddygiusepe/weave-intro-notebook/weave/objects/SystemPrompt/versions/y8wtlkPjQeSdGYvRN6F2N4RDXc8t76cuZESqXOsFv2M


ObjectRef(entity='eddygiusepe', project='weave-intro-notebook', name='SystemPrompt', _digest='y8wtlkPjQeSdGYvRN6F2N4RDXc8t76cuZESqXOsFv2M', _extra=())

## Model Tracking

Models are so common of an object type, that we have a special class to represent them: `weave.Model`. The only requirement is that we define a `predict` method.

![](https://github.com/wandb/weave/blob/media/intro/6.png?raw=1)

In [7]:
from openai import OpenAI

import weave

weave.init(PROJECT)


class OpenAIGrammarCorrector(weave.Model):
    # Properties are entirely user-defined
    openai_model_name: str
    system_message: str

    @weave.op()
    def predict(self, user_input):
        client = OpenAI()
        response = client.chat.completions.create(
            model=self.openai_model_name,
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": user_input},
            ],
            temperature=0,
        )
        return response.choices[0].message.content




corrector = OpenAIGrammarCorrector(
    openai_model_name="gpt-4o-mini",
    system_message="Você é um corretor gramatical, corrija a seguinte entrada do usuário.",
)

result = corrector.predict("     Eui           estouu indo para o cinema a               assistir um fi   lme!       ")
print(result)

🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf49-4991-7fd3-9ad7-5159b71c0dda
Eu estou indo para o cinema assistir a um filme!


## Dataset Tracking

Semelhante aos modelos, um objeto `weave.Dataset` existe para ajudar a rastrear, organizar e operar em conjuntos de dados

In [15]:
dataset = weave.Dataset(
    name="grammar-correction",
    rows=[
        {
            "user_input": "   Eui           estouu indo para o cinema a               assistir um fi   lme!   ",
            "expected": "Eu estou indo para o cinema a assistir um filme!",
        },
        {"user_input": "  Eu escrevo bom   ", "expected": "Eu escrevo bem"},
        {
            "user_input": "  GPT-3 é modelo de    IA mais    inteligente.   ",
            "expected": "GPT-3 é o modelo de IA mais inteligente.",
        },
    ],
)

weave.publish(dataset)

📦 Published to https://wandb.ai/eddygiusepe/weave-intro-notebook/weave/objects/grammar-correction/versions/H0X4QPw4JXlCG09rdaMR71zuWli63bJ5Rx0RbVNYi5I


ObjectRef(entity='eddygiusepe', project='weave-intro-notebook', name='grammar-correction', _digest='H0X4QPw4JXlCG09rdaMR71zuWli63bJ5Rx0RbVNYi5I', _extra=())

Notice that we saved a versioned `GrammarCorrector` object that captures the configurations you're experimenting with.

## Retrieve Published Objects & Ops

Você pode publicar objetos e então recuperá-los em seu código. Você pode até mesmo chamar funções de seus objetos recuperados!

In [16]:
import weave

weave.init(PROJECT)

corrector = OpenAIGrammarCorrector(
    openai_model_name="gpt-4o-mini",
    system_message="Você é um corretor gramatical, corrija a seguinte entrada do usuário.",
)

ref = weave.publish(corrector)
print(ref.uri())

📦 Published to https://wandb.ai/eddygiusepe/weave-intro-notebook/weave/objects/OpenAIGrammarCorrector/versions/AII0Pib0xuX2dnX7wUnASWJRgiT5qj4JQdok5Ei3Xzc
weave:///eddygiusepe/weave-intro-notebook/object/OpenAIGrammarCorrector:AII0Pib0xuX2dnX7wUnASWJRgiT5qj4JQdok5Ei3Xzc


In [17]:
ref.entity

'eddygiusepe'

In [18]:
ref.name

'OpenAIGrammarCorrector'

In [19]:
ref.digest

'AII0Pib0xuX2dnX7wUnASWJRgiT5qj4JQdok5Ei3Xzc'

In [20]:
import weave

weave.init(PROJECT)

# Note: this url is available from the UI after publishing the object!
ref_url = f"weave:///{ref.entity}/{PROJECT}/object/{ref.name}:{ref.digest}"
fetched_collector = weave.ref(ref_url).get()

# Notice: this object was loaded from remote location!
result = fetched_collector.predict("Eui           estouu indo para o cinema a               assistir um fi   lme!")

print(result)

🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf60-0cd0-7853-8bc7-96cb88fd2834
Eu estou indo para o cinema assistir a um filme!


# Evaluation

O desenvolvimento orientado a avaliação ajuda você a iterar de forma confiável em um aplicativo. A classe `Evaluation` é projetada para avaliar o desempenho de um `Model` em um dado `Dataset` ou conjunto de exemplos usando funções de pontuação.

Veja uma prévia da API abaixo:

In [21]:
import weave
from weave import Evaluation


# Defina qualquer função de pontuação personalizada: 
@weave.op()
def exact_match(expected: str, model_output: dict) -> dict:
    # Here is where you'd define the logic to score the model output
    return {"match": expected == model_output}


# Pontue seus exemplos usando funções de pontuação:
evaluation = Evaluation(
    dataset=dataset,  # can be a list of dictionaries or a weave.Dataset object
    scorers=[exact_match],  # can be a list of scoring functions
)

# Start tracking the evaluation
weave.init(PROJECT)
# Run the evaluation
summary = await evaluation.evaluate(corrector)  # can be a model or simple function

Evaluated 1 of 3 examples

Evaluated 2 of 3 examples

Evaluated 3 of 3 examples

Evaluation summary
{
    'exact_match': {'match': {'true_count': 1, 'true_fraction': 0.3333333333333333}},
    'model_latency': {'mean': 1.0510927836100261}
}

🍩 https://wandb.ai/eddygiusepe/weave-intro-notebook/r/call/0192cf60-11ae-7df1-b819-54244b1df2ea


## What's next?

Follow the [Build an Evaluation pipeline](http://wandb.me/weave_eval_tut) tutorial to learn more about Evaluation and begin iteratively improving your applications.